In [ ]:
# Dependencies
import requests
import json
import pandas as pd
from config import api_key
from pandas.io.json import json_normalize  

## Get Scopus Author Information using Multiple Scopus Author IDs

In [ ]:
# File to Load
radiation_oncology = "radiation_oncology.csv"

# Read the CSV file and store into Pandas DataFrame with the column Scopus Author ID as a string
radiation_oncology_df = pd.read_csv(radiation_oncology, encoding="utf-8",dtype={'Scopus Author ID': str})

#Change the column names to lower case with underscore for spaces
radiation_oncology_df.columns =  radiation_oncology_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
radiation_oncology_df.head()

In [ ]:
#List the column names in the dataframe
# radiation_oncology_df.columns

In [ ]:
#Check the data type in the dataframe columns called scopus_author_id and scopus_search
# radiation_oncology_df.scopus_author_id.dtype
# radiation_oncology_df.scopus_search.dtype

In [ ]:
#Save the column called scopus_author_id to a list called Author_ID_List
author_ID_List = radiation_oncology_df['scopus_author_id'].tolist()
print(author_ID_List)

In [ ]:
#Clean the Author_ID_List to remove nan
cleaned_Author_ID_List = [x for x in author_ID_List if str(x) != 'nan']
print(cleaned_Author_ID_List)

In [ ]:
#Add the necessary syntax to the cleaned_Author_ID_List
scopus_Mulitple_AuthorID_Query = []
for x in cleaned_Author_ID_List:
    authorID_string = "".join(("AU-ID(", x,")"))
    scopus_Mulitple_AuthorID_Query.append(authorID_string)
    
print(scopus_Mulitple_AuthorID_Query)

In [ ]:
# scopus_Search_List = radiation_oncology_df['scopus_search'].tolist()
# print(scopus_Search_List)

In [ ]:
# cleaned_Scopus_Search_List = [x for x in scopus_Search_List if str(x) != 'nan']
# print(cleaned_Scopus_Search_List)

In [ ]:
#https://dev.elsevier.com/guides/ScopusSearchViews.htm
#https://stackoverflow.com/questions/53558837/python-loop-to-pull-api-data-for-iterating-urls
#https://stackoverflow.com/questions/36410800/python-3-parse-json-from-multiple-api-requests-into-a-list-and-output-to-a-fil
#https://www.pluralsight.com/guides/web-scraping-with-request-python

multiple_author_list = []
multiple_author_dict = {}
outfilepath = "multiple_author_json.json"
keys = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]

def get_scopus_articles(scopus_Mulitple_AuthorID_Query):
    
    for authorid in scopus_Mulitple_AuthorID_Query:
        url = "http://api.elsevier.com/content/search/scopus?"
        fieldList = ["dc:identifier", "eid", "dc:title","prism:aggregationType", "subtype", "citedby-count",
                     "prism:publicationName","prism:volume","prism:issueIdentifier", "prism:pageRange", 
                     "prism:coverDate", "prism:doi","pubmed-id", "authid", "authname"]
                    
        headers = {
             "X-ELS-APIKey": api_key,
             'Accept':'application/json'
        }
        parameters = {
            "query": authorid,
            "field": ",".join(fieldList),
            "date": "2002-2003"
        }
        
        #Make the API request 
        single_author_response = requests.get(url, headers=headers, params=parameters)
        #print(single_author_response.url)
        #print(single_author_response.status_code)
        
              
        #Append each single_author_dict response to multiple_author_list to create a list of dictionaries
        single_author_dict = single_author_response.json()
        #print(type(single_author_dict)) 
        #print(single_author_dict)
        multiple_author_list.append(single_author_dict.copy())
    
    return multiple_author_list
       
get_scopus_articles(scopus_Mulitple_AuthorID_Query)



In [129]:
#https://stackoverflow.com/questions/48177934/flatten-or-unpack-list-of-nested-dicts-in-dataframe
#https://stackoverflow.com/questions/50161070/convert-list-of-dicts-of-dict-into-dataframe


def make_scopus_articles_df(multiple_author_list):
    #final_list = json_normalize(multiple_author_list, meta=["search-results"], record_path=["search-results", "entry"])
    final_df = pd.DataFrame.from_dict(json_normalize(multiple_author_list, meta=["search-results"], record_path=["search-results", "entry"]),orient="columns")
    return final_df

# df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')
# df
make_scopus_articles_df(multiple_author_list)

,@_fa,author,citedby-count,dc:identifier,dc:title,eid,error,prism:aggregationType,prism:coverDate,prism:doi,prism:issueIdentifier,prism:pageRange,prism:publicationName,prism:url,prism:volume,pubmed-id,subtype,subtypeDescription,search-results
0,true,NaN,NaN,NaN,NaN,NaN,Result set was empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'opensearch:totalResults': '0', 'opensearch:s..."
1,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",6,SCOPUS_ID:0042128695,Targeting colon cancer cells with genistein-17...,2-s2.0-0042128695,NaN,Journal,2003-01-01,NaN,5,955-959,International journal of oncology,https://api.elsevier.com/content/abstract/scop...,22,12684659,ar,Article,"{'opensearch:totalResults': '1', 'opensearch:s..."
2,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",157,SCOPUS_ID:9144261127,Geldanamycin and 17-Allylamino-17-demethoxygel...,2-s2.0-9144261127,NaN,Journal,2003-12-15,NaN,24,8984-8995,Cancer Research,https://api.elsevier.com/content/abstract/scop...,63,14695217,ar,Article,"{'opensearch:totalResults': '15', 'opensearch:..."
3,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",6,SCOPUS_ID:0347949519,Inhibition of cyclooxygenase-2 with NS-398 and...,2-s2.0-0347949519,NaN,Journal,2003-11-01,10.1080/09553000310001621400,11,879-888,International Journal of Radiation Biology,https://api.elsevier.com/content/abstract/scop...,79,14698956,ar,Article,"{'opensearch:totalResults': '15', 'opensearch:..."
4,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",26,SCOPUS_ID:0042383099,Those in gene therapy should pay closer attent...,2-s2.0-0042383099,NaN,Journal,2003-10-01,10.1016/S0360-3016(03)00421-8,2,597-599,International Journal of Radiation Oncology Bi...,https://api.elsevier.com/content/abstract/scop...,57,12957278,le,Letter,"{'opensearch:totalResults': '15', 'opensearch:..."
5,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",113,SCOPUS_ID:0038407274,2-Deoxy-D-glucose-induced cytotoxicity and rad...,2-s2.0-0038407274,NaN,Journal,2003-06-15,NaN,12,3413-3417,Cancer Research,https://api.elsevier.com/content/abstract/scop...,63,12810678,ar,Article,"{'opensearch:totalResults': '15', 'opensearch:..."
6,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",2,SCOPUS_ID:0037215656,The holy grail of radiation oncology: Lessons ...,2-s2.0-0037215656,NaN,Journal,2003-01-01,10.1016/S0360-3016(02)03861-0,1,3-4,International Journal of Radiation Oncology Bi...,https://api.elsevier.com/content/abstract/scop...,55,12504029,ed,Editorial,"{'opensearch:totalResults': '15', 'opensearch:..."
7,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",0,SCOPUS_ID:0037262512,Clinical Trials Referral Resource. Clinical tr...,2-s2.0-0037262512,NaN,Journal,2003-01-01,NaN,1,84-89,"Oncology (Williston Park, N.Y.)",https://api.elsevier.com/content/abstract/scop...,17,12599933,ar,Article,"{'opensearch:totalResults': '15', 'opensearch:..."
8,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",27,SCOPUS_ID:0037216745,Inhibition of stress-inducible kinase pathways...,2-s2.0-0037216745,NaN,Journal,2003-01-01,10.1128/MCB.23.1.322-334.2003,1,322-334,Molecular and Cellular Biology,https://api.elsevier.com/content/abstract/scop...,23,12482984,ar,Article,"{'opensearch:totalResults': '15', 'opensearch:..."
9,true,"[{'@_fa': 'true', 'author-url': 'https://api.e...",2,SCOPUS_ID:0037032514,Treatment of nasopharyngeal cancer: Raising th...,2-s2.0-0037032514,NaN,Journal,2002-11-06,NaN,21,1594-1595,Journal of the National Cancer Institute,https://api.elsevier.com/content/abstract/scop...,94,12419779,ed,Editorial,"{'opensearch:totalResults': '15', 'opensearch:..."


In [132]:
#https://stackoverflow.com/questions/29325458/dictionary-column-in-pandas-dataframe/29330853
def fix_dataframe(final_df):
    pd.concat([final_df.drop(['search-results'], axis=1), final_df['search-results'].apply(pd.Series)], axis=1)
#pd.concat([dw.drop(['ColC'], axis=1), dw['ColC'].apply(pd.Series)], axis=1)

fix_dataframe(final_df)

NameError: name 'final_df' is not defined